# Interacción con APIs

Ahora que sabemos lo básico de `requests` veamos cómo podemos interactuar con nuestra API de clientes.

Si no la tenemos en marcha vamos a activarla, recordad que le cambiamos el nombre así que deberíamos cambiar el script en `Pipfile`:

```toml
[scripts]
api = "uvicorn main:app --reload"
```

Y luego:

```bash
$ pipenv run api
```

Alternativamente podemos utilizar la API desplegada en `Deta`.

## Listar los clientes

Empecemos con una petición para consultar todos los clientes:

In [1]:
import requests

dominio = "http://127.0.0.1:8000"
# dominio = "https://vlavj0.deta.dev"
        
r = requests.get(f"{dominio}/clientes")

print(r.text)

[{"dni":"15J","nombre":"Marta","apellido":"Pérez"},{"dni":"48H","nombre":"Manolo","apellido":"López"},{"dni":"28Z","nombre":"Mariana","apellido":"García"}]


Recordemos que nuestra api devuelve respuestas de tipo `json` así que debemos transformarlas a algo que podamos manipular. La propia respuesta tiene un método llamado `json()` que transforma el contenido en un diccionario:

In [2]:
r.json()

[{'dni': '15J', 'nombre': 'Marta', 'apellido': 'Pérez'},
 {'dni': '48H', 'nombre': 'Manolo', 'apellido': 'López'},
 {'dni': '28Z', 'nombre': 'Mariana', 'apellido': 'García'}]

Podemos iterar cada valor de la colección cómodamente:

In [3]:
for cliente in r.json():
    print(cliente['dni'], cliente['nombre'], cliente['apellido'])

15J Marta Pérez
48H Manolo López
28Z Mariana García


## Consultar un cliente

¿Fácil verdad? Vamos a consultar un cliente a partir de su DNI:

In [4]:
dni = "48H"

r = requests.get(f"{dominio}/clientes/buscar/{dni}")
print(r.text)

{"dni":"48H","nombre":"Manolo","apellido":"López"}


En esta ocasión en lugar de una lista devolvemos solo un objeto, podemos simplemente acceder a sus campos:

In [5]:
cliente = r.json()

print(cliente['dni'], cliente['nombre'], cliente['apellido'])

48H Manolo López


En caso de que el cliente no se encuentre devolvemos un error 404, ya vimos como podemos tratar:



In [6]:
import requests
from requests import HTTPError

dominio = "http://127.0.0.1:8000"
dni = "99Z"
    
try:
    r = requests.get(f"{dominio}/clientes/buscar/{dni}")
    r.raise_for_status()
    r.json()
    print(cliente['dni'], cliente['nombre'], cliente['apellido'])
except HTTPError as ex:
    print(ex)

404 Client Error: Not Found for url: http://127.0.0.1:8000/clientes/buscar/99Z/


## Crear un cliente

Para la creación establecimos el uso del método POST con unos argumentos, los valores ya vimos que se pueden enviar en un `payload` en forma de diccionario clave-valor:

In [7]:
import requests
from requests import HTTPError

dominio = "http://127.0.0.1:8000"

payload = {
    'dni': '99Z',
    'nombre': 'Hector',
    'apellido': 'Costa'
}
    
r = requests.post(f"{dominio}/clientes/crear/", data=payload)
print(r.text)

{"detail":[{"loc":["body"],"msg":"value is not a valid dict","type":"type_error.dict"}]}


Como vemos nos devuelve un error, nuestra API nos dice que el tipo diccionario del `payload` con los datos enviado no es válido. La razón es que nuestra API solo acepta datos en formato JSON así que cambiaremos el tipo de envío:

In [8]:
r = requests.post(f"{dominio}/clientes/crear/", json=payload)
print(r.text)

{"dni":"99Z","nombre":"Hector","apellido":"Costa"}


Ahora sí ya tenemos el usuario creado, en caso de crearlo de nuevo recibiremos un error por DNI duplicado:

In [9]:
r = requests.post(f"{dominio}/clientes/crear/", json=payload)
print(r.text)

{"detail":[{"loc":["body","dni"],"msg":"Cliente ya existente o DNI incorrecto","type":"value_error"}]}


Podemos listar todos los clientes:

In [10]:
r = requests.get(f"{dominio}/clientes")

for cliente in r.json():
    print(cliente['dni'], cliente['nombre'], cliente['apellido'])

15J Marta Pérez
48H Manolo López
28Z Mariana García
99Z Hector Costa


## Actualizar un cliente

Como era de esperar `requests` contiene otros métodos como `put` y `delete`, vamos a modificar un cliente:

In [11]:
import requests
from requests import HTTPError

dominio = "http://127.0.0.1:8000"

payload = {
    'dni': '99Z',
    'nombre': 'Victor',
    'apellido': 'Acosta'
}
    
r = requests.put(f"{dominio}/clientes/actualizar/", json=payload)
print(r.text)

{"dni":"99Z","nombre":"Victor","apellido":"Acosta"}


Podemos volver a consultar la lista completo de clientes a ver si todo está correcto:

In [12]:
r = requests.get(f"{dominio}/clientes")

for cliente in r.json():
    print(cliente['dni'], cliente['nombre'], cliente['apellido'])

15J Marta Pérez
48H Manolo López
28Z Mariana García
99Z Victor Acosta


## Borrar un cliente

Por último vamos a borrar el cliente creado mediante el método `delete` pasándole el DNI:

In [13]:
import requests
from requests import HTTPError

dominio = "http://127.0.0.1:8000"
dni = '99Z'
    
r = requests.delete(f"{dominio}/clientes/borrar/{dni}")
print(r.text)

{"dni":"99Z","nombre":"Victor","apellido":"Acosta"}


En principio debería haberse borrado, vamos a confirmarlo recuperando la lista completa de clientes:

In [14]:
r = requests.get(f"{dominio}/clientes")

for cliente in r.json():
    print(cliente['dni'], cliente['nombre'], cliente['apellido'])

15J Marta Pérez
48H Manolo López
28Z Mariana García


¡Perfecto! Y si intentamos borrar un cliente que no existe...

In [15]:
dni = '99Z'
    
r = requests.delete(f"{dominio}/clientes/borrar/{dni}")
print(r.text)

{"detail":"Not Found"}


Pues nos devuelve un error 404 tal y como lo programamos en la API:

In [16]:
r.status_code

404

Muy bien, con esto hemos aprendido a crear una API e interactuar con ella a través de código.